<a href="https://colab.research.google.com/github/ariefwcks303/MachineLearningFromScratch/blob/main/Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine

In [2]:
data = load_wine()

X, y = data.data, data.target

In [3]:
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


Dataset ini memiliki 178 baris dan 13 kolom fitur independen serta satu kolom dependen (target)

##Splitting Data

In [4]:
# Pembagian data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Fitur Methods

In [11]:
# ------------------- Filter Methods -------------------
# Menggunakan SelectKBest
filter_selector = SelectKBest(score_func=chi2, k=3)

X_train_filter = filter_selector.fit_transform(X_train, y_train)
X_test_filter = filter_selector.transform(X_test)

print("Fitur terpilih:", filter_selector.get_support(indices=True))

Fitur terpilih: [ 6  9 12]


##Wrapper Methods



In [12]:
# Menggunakan RFE (Recursive Feature Elimination)
model = LogisticRegression(solver='lbfgs', max_iter=5000)
rfe_selector = RFE(model, n_features_to_select=3)
X_train_rfe = rfe_selector.fit_transform(X_train, y_train)
X_test_rfe = rfe_selector.transform(X_test)

print("Fitur Terpilih: ", rfe_selector.get_support(indices=True))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitur Terpilih:  [ 0  6 11]


##Embeded Methods

In [14]:
# ------------------- Embedded Methods -------------------
# Menggunakan Random Forest untuk mendapatkan fitur penting
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

threshold = 0.05
important_features_indices = [i for i in range(len(importances)) if importances[i] > threshold]

X_important_train = X_train[:, important_features_indices]
X_important_test = X_test[:, important_features_indices]

In [15]:
print("Fitur yang dipilih dengan Embedded Methods (di atas ambang batas):")

for i in important_features_indices:
    print(f"{data.feature_names[i]}: {importances[i]}")

print("\nDimensi data pelatihan dengan fitur penting: ", X_important_train.shape)
print("Dimensi data pengujian dengan fitur penting: ", X_important_test.shape)


Fitur yang dipilih dengan Embedded Methods (di atas ambang batas):
alcohol: 0.11239773542143086
flavanoids: 0.20229341635663622
color_intensity: 0.1712021830864957
hue: 0.07089132259413944
od280/od315_of_diluted_wines: 0.1115643167260497
proline: 0.13904586955351153

Dimensi data pelatihan dengan fitur penting:  (142, 6)
Dimensi data pengujian dengan fitur penting:  (36, 6)


In [16]:
# Evaluasi dengan fitur terpilih dari masing-masing metode

def evaluate_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    return accuracy

In [19]:
# Model Logistic Regression untuk Filter Methods
logistic_model_filter = LogisticRegression(max_iter=200)
accuracy_filter = evaluate_model(X_train_filter, X_test_filter, y_train, y_test, logistic_model_filter)


# Model Logistic Regression untuk Wrapper Methods
logistic_model_rfe = LogisticRegression(max_iter=200)
accuracy_rfe = evaluate_model(X_train_rfe, X_test_rfe, y_train, y_test, logistic_model_rfe)


# Model Random Forest untuk Embedded Methods
accuracy_rf = evaluate_model(X_important_train, X_important_test, y_train, y_test, rf_model) # Corrected typo



print(f"\n Akurasi Model dengan Filter Methods : {accuracy_filter:.4f}")
print(f"\n Akurasi Model dengan Wrapper Methods : {accuracy_rfe:.4f}")
print(f"\n Akurasi Model dengan Embedded Methods : {accuracy_rf:.4f}")

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 Akurasi Model dengan Filter Methods : 0.9722

 Akurasi Model dengan Wrapper Methods : 0.9444

 Akurasi Model dengan Embedded Methods : 1.0000


Seperti yang dapat Anda lihat masing-masing metode memiliki performa yang berbeda. Hal ini karena penggunaan fitur yang berbeda pada proses pelatihan model machine learning. Karena embedded methods memiliki akurasi sempurna, mungkin kita berpikir “Ya sudah kita pakai embedded methods saja untuk semua kasus.” Eiitss, tidak semudah itu kawan, meskipun embedded methods bisa digunakan dalam berbagai skenario, tetapi tidak semua masalah cocok menggunakan metode ini. Ada beberapa hal yang menjadi pertimbangan ketika kita akan melakukan feature selection.

- Jenis Model: tidak semua algoritma mendukung embedded methods, contohnya linear regression standar atau k-nearest neighbors (KNN) karena algoritma tersebut tidak memiliki mekanisme bawaan untuk memilih fitur. Untuk model-model tersebut, kita perlu menggunakan feature selection secara independen (seperti filter atau wrapper methods).

- Ukuran dan Kompleksitas Data: jika data sangat besar atau kompleks, embedded feature selection bisa memakan waktu dan sumber daya komputasi yang lebih besar. Dalam beberapa kasus, menggunakan teknik lain seperti filter methods untuk mereduksi dimensi data di awal bisa lebih efisien.

- Kebutuhan Interpretabilitas: beberapa model dengan embedded feature selection (misalnya, Random Forest) bisa menghasilkan model yang sulit diinterpretasi. Jika interpretabilitas sangat penting, gunakan teknik lain yang lebih sesuai.

- Tipe Fitur: jika fitur yang ada memiliki tipe yang sangat berbeda-beda misalnya numerik dan kategorikal, embedded feature selection dalam model tertentu mungkin tidak bekerja dengan baik.


Jadi semua metode di atas memiliki kelebihannya masing-masing, ketika kelak kita menggunakan data yang lebih kompleks serta fitur yang lebih banyak, belum tentu embedded methods menjadi pilihan terbaik. Karena pada dasarnya, pembangunan machine learning tidak hanya bergantung kepada performa, tetapi juga efisiensi biaya dan optimisasi bisnis.